# App-8-MiniZinc : Modelisation declarative par contraintes

**Navigation** : [<< App-7 Wordle](App-7-Wordle.ipynb) | [Index](../README.md) | [App-9 EdgeDetection >>](App-9-EdgeDetection.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. **Comprendre** la philosophie de MiniZinc : decrire un probleme de maniere declarative
2. **Ecrire** des modeles MiniZinc (variables, contraintes, objectifs, sorties)
3. **Utiliser** les contraintes globales (`alldifferent`, `cumulative`, `circuit`, `table`)
4. **Comparer** les approches imperative (CP-SAT) et declarative (MiniZinc) sur un meme probleme
5. **Piloter** MiniZinc depuis Python (package `minizinc` ou ligne de commande)

### Prerequis
- Programmation par contraintes avancee ([Search-8-CSP-Advanced](../Foundations/Search-8-CSP-Advanced.ipynb))
- Notions de contraintes globales (AllDifferent, cumulative)
- Python 3.10+

### Duree estimee : 50 minutes

### Liens avec d'autres notebooks
- [App-5-Timetabling](App-5-Timetabling.ipynb) : emploi du temps avec MiniZinc + OR-Tools
- [App-3-NurseScheduling](App-3-NurseScheduling.ipynb) et [App-4-JobShopScheduling](App-4-JobShopScheduling.ipynb) : exemples de CP imperatif
- Serie [Sudoku](../../Sudoku/README.md) : solveurs CSP complets

---

## 1. Introduction : pourquoi MiniZinc ? (~5 min)

Dans les notebooks precedents, nous avons resolu des CSP en Python de maniere **imperative** : nous avons cree des variables, ajoute des contraintes une par une, configure un solveur, puis lance la recherche. C'est l'approche utilisee par OR-Tools/CP-SAT, python-constraint, etc.

**MiniZinc** adopte une approche radicalement differente : l'approche **declarative**.

| Aspect | Imperatif (CP-SAT, Python) | Declaratif (MiniZinc) |
|--------|---------------------------|----------------------|
| Philosophie | Decrire **COMMENT** resoudre | Decrire **QUOI** resoudre |
| Solveur | Choisi au codage | Interchangeable (Gecode, Chuffed, OR-Tools...) |
| Syntaxe | API Python/Java/C++ | Langage dedie, proche des maths |
| Separation modele/donnees | Melangees dans le code | Modele `.mzn` + donnees `.dzn` |
| Reutilisabilite | Refactoring necessaire | Changer le fichier de donnees suffit |

### MiniZinc en bref

- **Langage de modelisation** de haut niveau pour problemes combinatoires
- **Standard de l'industrie** en programmation par contraintes (depuis 2007, Universite de Melbourne)
- **Compile** les modeles vers FlatZinc (format intermediaire) puis delegue a un solveur
- **Ecosysteme** : IDE dedie, bibliotheque de contraintes globales, challenges MiniZinc annuels
- **Solveurs compatibles** : Gecode, Chuffed, OR-Tools, COIN-BC, CPLEX, Gurobi...

### Architecture

```
Modele (.mzn)  +  Donnees (.dzn)  -->  Compilateur MiniZinc  -->  FlatZinc  -->  Solveur
                                                                               (Gecode, Chuffed, ...)
                                                                                  |
                                                                              Solution
```

Dans ce notebook, nous allons ecrire des modeles MiniZinc sous forme de chaines Python, les expliquer pas a pas, et (si le package `minizinc` est disponible) les executer directement.

In [ ]:
# Imports pour tout le notebook
import sys
import time
import textwrap
import subprocess
import shutil
import matplotlib.pyplot as plt
import numpy as np

# Verifier la disponibilite de MiniZinc
MINIZINC_AVAILABLE = False
MINIZINC_CLI_AVAILABLE = shutil.which('minizinc') is not None

try:
    import minizinc
    MINIZINC_AVAILABLE = True
    print(f"Package minizinc disponible (version {minizinc.__version__})")
except ImportError:
    print("Package minizinc non installe (pip install minizinc)")
    print("Les modeles seront affiches mais pas executes via l'API Python.")

if MINIZINC_CLI_AVAILABLE:
    print(f"CLI minizinc disponible : {shutil.which('minizinc')}")
else:
    print("CLI minizinc non trouvee dans le PATH.")
    print("Les modeles seront affiches a titre illustratif.")

# Verification OR-Tools pour les comparaisons
ORTOOLS_AVAILABLE = False
try:
    from ortools.sat.python import cp_model
    ORTOOLS_AVAILABLE = True
    print("OR-Tools CP-SAT disponible pour les comparaisons.")
except ImportError:
    print("OR-Tools non installe (pip install ortools).")

print("\nImports OK")

Definissons des fonctions utilitaires pour afficher et (optionnellement) executer des modeles MiniZinc depuis Python.

In [ ]:
def display_model(model_str, title="Modele MiniZinc"):
    """Affiche un modele MiniZinc avec numerotation des lignes."""
    print(f"\n{'=' * 60}")
    print(f"  {title}")
    print(f"{'=' * 60}")
    lines = textwrap.dedent(model_str).strip().split('\n')
    for i, line in enumerate(lines, 1):
        print(f"  {i:2d} | {line}")
    print(f"{'=' * 60}\n")


def run_minizinc_model(model_str, solver_name="gecode", timeout_sec=30, data_str=None):
    """Execute un modele MiniZinc si le package est disponible.
    
    Retourne le resultat ou None si non disponible.
    """
    if MINIZINC_AVAILABLE:
        try:
            model = minizinc.Model()
            model.add_string(textwrap.dedent(model_str).strip())
            if data_str:
                model.add_string(textwrap.dedent(data_str).strip())
            solver = minizinc.Solver.lookup(solver_name)
            instance = minizinc.Instance(solver, model)
            import datetime
            result = instance.solve(timeout=datetime.timedelta(seconds=timeout_sec))
            return result
        except Exception as e:
            print(f"Erreur d'execution MiniZinc : {e}")
            return None
    elif MINIZINC_CLI_AVAILABLE:
        # Fallback : ecrire dans un fichier temporaire et appeler le CLI
        import tempfile, os
        try:
            with tempfile.NamedTemporaryFile(mode='w', suffix='.mzn', delete=False) as f:
                f.write(textwrap.dedent(model_str).strip())
                mzn_path = f.name
            cmd = ['minizinc', '--solver', solver_name, mzn_path]
            proc = subprocess.run(cmd, capture_output=True, text=True, timeout=timeout_sec)
            os.unlink(mzn_path)
            if proc.returncode == 0:
                print("Sortie MiniZinc :")
                print(proc.stdout)
            else:
                print(f"Erreur : {proc.stderr}")
            return proc.stdout
        except Exception as e:
            print(f"Erreur CLI MiniZinc : {e}")
            return None
    else:
        print("[MiniZinc non disponible - modele affiche uniquement]")
        return None


print("Fonctions utilitaires definies.")

---

## 2. Syntaxe de base de MiniZinc (~10 min)

Un modele MiniZinc se compose de quatre elements principaux :

| Element | Syntaxe | Role |
|---------|---------|------|
| **Variables de decision** | `var 1..9: x;` | Ce que l'on cherche |
| **Contraintes** | `constraint x != y;` | Ce que l'on impose |
| **Objectif** | `solve satisfy;` ou `solve minimize obj;` | Satisfaction ou optimisation |
| **Sortie** | `output ["x=\(x), y=\(y)"];` | Format d'affichage |

### Types de donnees

| Type | Exemple | Description |
|------|---------|-------------|
| `int` | `int: n = 8;` | Entier fixe (parametre) |
| `var int` | `var 1..n: x;` | Variable de decision entiere |
| `bool` | `var bool: b;` | Variable booleenne |
| `float` | `var 0.0..1.0: p;` | Variable flottante |
| `array` | `array[1..n] of var 1..n: q;` | Tableau de variables |
| `set` | `var set of 1..5: s;` | Ensemble variable |

### Premier exemple : systeme d'equations

Commencons par un probleme tres simple : trouver $x$ et $y$ tels que :
$$x + y = 10 \quad \text{et} \quad x \times y = 21$$

En Python, on pourrait resoudre cela algebriquement. En MiniZinc, on le **declare** simplement.

In [ ]:
# Premier modele MiniZinc : systeme d'equations
model_equations = """
% Systeme d'equations : x + y = 10, x * y = 21
var 1..10: x;
var 1..10: y;

constraint x + y = 10;
constraint x * y = 21;

solve satisfy;

output ["x = ", show(x), ", y = ", show(y), "\\n"];
"""

display_model(model_equations, "Systeme d'equations")

# Execution si disponible
result = run_minizinc_model(model_equations)
if result is None and not MINIZINC_CLI_AVAILABLE:
    print("Resultat attendu : x = 3, y = 7 (ou x = 7, y = 3)")
    print("Verification : 3 + 7 = 10, 3 * 7 = 21")

### Interpretation : premier modele

**Sortie obtenue** : le solveur trouve $x = 3, y = 7$ (ou l'inverse).

| Ligne | Element | Explication |
|-------|---------|-------------|
| `var 1..10: x;` | Declaration | $x$ est une variable de decision dans $\{1, ..., 10\}$ |
| `constraint x + y = 10;` | Contrainte | Egalite (= et non ==) |
| `solve satisfy;` | Objectif | Trouver une solution faisable (pas d'optimisation) |
| `output [...]` | Sortie | Formatage avec `show()` pour les variables |

**Points cles** :
1. Le modele est **auto-explicatif** : on lit directement les equations mathematiques
2. Le `%` introduit un commentaire (comme `#` en Python)
3. Chaque instruction se termine par `;`
4. On ne specifie pas d'algorithme de resolution : le solveur s'en charge

### Exemple avec optimisation : probleme de monnaie

Trouver le nombre minimal de pieces (1c, 5c, 10c, 25c) pour rendre exactement 99 centimes.

In [ ]:
model_coins = """
% Probleme de monnaie : rendre 99 centimes avec le minimum de pieces
var 0..99: p1;   % pieces de 1 centime
var 0..19: p5;   % pieces de 5 centimes
var 0..9:  p10;  % pieces de 10 centimes
var 0..3:  p25;  % pieces de 25 centimes

% Contrainte : la somme vaut exactement 99
constraint p1 + 5*p5 + 10*p10 + 25*p25 = 99;

% Objectif : minimiser le nombre total de pieces
var int: total = p1 + p5 + p10 + p25;
solve minimize total;

output ["Pieces: ", show(p25), "x25c + ", show(p10), "x10c + ",
        show(p5), "x5c + ", show(p1), "x1c = 99c\\n",
        "Total: ", show(total), " pieces\\n"];
"""

display_model(model_coins, "Probleme de monnaie (optimisation)")

result = run_minizinc_model(model_coins)
if result is None and not MINIZINC_CLI_AVAILABLE:
    print("Resultat attendu : 3x25c + 2x10c + 0x5c + 4x1c = 99c")
    print("Total : 9 pieces (optimal)")

### Interpretation : optimisation

**Sortie obtenue** : 3 pieces de 25c + 2 pieces de 10c + 4 pieces de 1c = 99c, soit 9 pieces au total.

| Element | Syntaxe MiniZinc | Description |
|---------|------------------|-------------|
| Variable derivee | `var int: total = ...;` | Variable calculee a partir d'autres |
| Optimisation | `solve minimize total;` | Le solveur cherche la solution optimale |

**Points cles** :
1. Passer de `solve satisfy` a `solve minimize/maximize` transforme un CSP en COP (Constraint Optimization Problem)
2. Le solveur explore l'espace et garantit l'optimalite (contrairement aux heuristiques)
3. Le modele reste compact et lisible

---

## 3. Contraintes globales dans MiniZinc (~10 min)

La puissance de MiniZinc reside en grande partie dans sa bibliotheque de **contraintes globales**. Ces contraintes encapsulent des patterns recurrents de modelisation et permettent au solveur d'utiliser des algorithmes de propagation specialises.

### Principales contraintes globales

| Contrainte | Syntaxe | Description | Application typique |
|------------|---------|-------------|---------------------|
| `alldifferent` | `alldifferent(array)` | Toutes les valeurs distinctes | Sudoku, coloration |
| `cumulative` | `cumulative(start, dur, res, limit)` | Ressource cumulative limitee | Ordonnancement |
| `circuit` | `circuit(array)` | Circuit hamiltonien | TSP |
| `table` | `table(vars, tuples)` | Combinaisons autorisees | Contraintes extensionnelles |
| `count` | `count(array, val, n)` | Nombre d'occurrences | Frequences |
| `element` | `element(i, array, val)` | Indexation par variable | Acces indirect |
| `increasing` | `increasing(array)` | Ordre croissant | Symetries |

Pour utiliser les contraintes globales, il faut inclure la bibliotheque : `include "globals.mzn";`

### Exemple phare : N-Reines en MiniZinc

Le probleme des N-Reines illustre parfaitement la concision de MiniZinc. Rappel : placer $N$ reines sur un echiquier $N \times N$ sans qu'aucune ne menace une autre.

In [ ]:
# N-Reines en MiniZinc : modele complet en 7 lignes utiles
model_nqueens = """
include "globals.mzn";

int: n = 8;
array[1..n] of var 1..n: q;  % q[i] = ligne de la reine en colonne i

constraint alldifferent(q);                        % pas meme ligne
constraint alldifferent([q[i] + i | i in 1..n]);   % pas meme diagonale montante
constraint alldifferent([q[i] - i | i in 1..n]);   % pas meme diagonale descendante

solve satisfy;

output [if fix(q[j]) == i then "Q " else ". " endif ++
        if j == n then "\\n" else "" endif
        | i in 1..n, j in 1..n];
"""

display_model(model_nqueens, "N-Reines (MiniZinc)")

result = run_minizinc_model(model_nqueens)
if result is None and not MINIZINC_CLI_AVAILABLE:
    print("Resultat attendu : un echiquier 8x8 avec 8 reines valides.")
    # Affichons une solution connue
    known = [1, 5, 8, 6, 3, 7, 2, 4]  # q[col] = row
    for row in range(1, 9):
        line = ""
        for col in range(8):
            line += "Q " if known[col] == row else ". "
        print(line)

### Interpretation : N-Reines declaratif

**Sortie obtenue** : une solution valide du probleme des 8-Reines.

| Ligne du modele | Explication |
|-----------------|-------------|
| `include "globals.mzn";` | Charge la bibliotheque de contraintes globales |
| `array[1..n] of var 1..n: q;` | Tableau de $n$ variables, chacune dans $\{1..n\}$ |
| `alldifferent(q)` | Toutes les reines sur des lignes differentes |
| `alldifferent([q[i]+i \| i in 1..n])` | Diagonales montantes toutes differentes |
| `alldifferent([q[i]-i \| i in 1..n])` | Diagonales descendantes toutes differentes |

**Comparaison avec le modele Python** :

Le modele MiniZinc tient en **3 contraintes** contre une boucle imbriquee en Python. La comprehension de tableau `[q[i]+i | i in 1..n]` est l'equivalent MiniZinc des list comprehensions Python.

> **Point cle** : `alldifferent` n'est pas une simple boucle de `!=`. Le solveur utilise des algorithmes de propagation specialises (filtrage par couplage maximum dans un graphe biparti) qui sont bien plus efficaces que des contraintes binaires individuelles.

### Comparaison directe : N-Reines en CP-SAT vs MiniZinc

Pour mesurer la difference de concision, implementons le meme probleme en CP-SAT (OR-Tools).

In [ ]:
# N-Reines en CP-SAT (OR-Tools) - approche imperative
def solve_nqueens_cpsat(n):
    """Resout N-Reines avec OR-Tools CP-SAT."""
    if not ORTOOLS_AVAILABLE:
        print("OR-Tools non disponible, affichage du code uniquement.")
        return None, 0
    
    model = cp_model.CpModel()
    
    # Variables : q[i] = ligne de la reine en colonne i
    q = [model.NewIntVar(0, n - 1, f'q_{i}') for i in range(n)]
    
    # Contrainte : toutes sur des lignes differentes
    model.AddAllDifferent(q)
    
    # Contrainte : diagonales differentes
    diag1 = [q[i] + i for i in range(n)]  # diagonale montante
    diag2 = [q[i] - i for i in range(n)]  # diagonale descendante
    model.AddAllDifferent(diag1)
    model.AddAllDifferent(diag2)
    
    # Resolution
    solver = cp_model.CpSolver()
    start = time.time()
    status = solver.Solve(model)
    elapsed = time.time() - start
    
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        solution = [solver.Value(q[i]) for i in range(n)]
        return solution, elapsed
    return None, elapsed


# Comparaison de concision
cpsat_code = '''
# CP-SAT (Python, OR-Tools) - 12 lignes utiles
model = cp_model.CpModel()
q = [model.NewIntVar(0, n-1, f'q_{i}') for i in range(n)]
model.AddAllDifferent(q)
model.AddAllDifferent([q[i] + i for i in range(n)])
model.AddAllDifferent([q[i] - i for i in range(n)])
solver = cp_model.CpSolver()
status = solver.Solve(model)
solution = [solver.Value(q[i]) for i in range(n)]
'''

mzn_code = '''
% MiniZinc - 5 lignes utiles
include "globals.mzn";
int: n = 8;
array[1..n] of var 1..n: q;
constraint alldifferent(q);
constraint alldifferent([q[i]+i | i in 1..n]);
constraint alldifferent([q[i]-i | i in 1..n]);
solve satisfy;
'''

print("Comparaison de concision : N-Reines")
print("=" * 50)
print(f"CP-SAT (Python) : ~12 lignes utiles")
print(f"MiniZinc        : ~5 lignes utiles")
print(f"Ratio           : MiniZinc est ~2.4x plus concis")

# Execution CP-SAT
if ORTOOLS_AVAILABLE:
    sol_cpsat, t_cpsat = solve_nqueens_cpsat(8)
    print(f"\nCP-SAT solution  : {sol_cpsat}")
    print(f"CP-SAT temps     : {t_cpsat*1000:.2f} ms")

### Interpretation : concision et lisibilite

| Critere | CP-SAT (Python) | MiniZinc |
|---------|-----------------|----------|
| Lignes de code utiles | ~12 | ~5 |
| Boilerplate (imports, solver, extraction) | Oui | Non |
| Lisibilite mathematique | Moyenne | Haute |
| Flexibilite du solveur | CP-SAT uniquement | Gecode, Chuffed, OR-Tools... |
| Debug et introspection | API riche | Plus limite |

**Points cles** :
1. MiniZinc supprime tout le boilerplate (creation de solveur, extraction de valeurs)
2. La syntaxe MiniZinc est plus proche de la notation mathematique
3. En contrepartie, CP-SAT offre plus de controle programmatique (callbacks, branchement personnalise)

### Autres contraintes globales en action

Voyons `cumulative` pour l'ordonnancement et `circuit` pour le TSP.

In [ ]:
# Exemple cumulative : ordonnancement simple
model_scheduling = """
include "globals.mzn";

% 4 taches avec durees et besoins en ressource
int: n_tasks = 4;
int: horizon = 20;
int: capacity = 3;  % capacite maximale de la ressource

array[1..n_tasks] of int: duration = [5, 3, 4, 6];
array[1..n_tasks] of int: resource = [2, 1, 2, 1];

% Variables : debut de chaque tache
array[1..n_tasks] of var 0..horizon: start;

% Contrainte cumulative : a tout instant, la somme des ressources <= capacite
constraint cumulative(start, duration, resource, capacity);

% Objectif : minimiser le makespan (date de fin de la derniere tache)
var int: makespan = max([start[i] + duration[i] | i in 1..n_tasks]);
solve minimize makespan;

output ["Tache ", show(i), " : debut=", show(start[i]),
        " fin=", show(start[i] + duration[i]), "\\n" | i in 1..n_tasks]
       ++ ["Makespan = ", show(makespan), "\\n"];
"""

display_model(model_scheduling, "Ordonnancement avec cumulative")

result = run_minizinc_model(model_scheduling)
if result is None and not MINIZINC_CLI_AVAILABLE:
    print("Resultat attendu :")
    print("  Tache 1 (dur=5, res=2) : debut=0,  fin=5")
    print("  Tache 2 (dur=3, res=1) : debut=0,  fin=3")
    print("  Tache 3 (dur=4, res=2) : debut=5,  fin=9")
    print("  Tache 4 (dur=6, res=1) : debut=3,  fin=9")
    print("  Makespan = 9")

### Interpretation : contrainte cumulative

**Sortie obtenue** : un ordonnancement optimal avec makespan minimise.

| Parametre de `cumulative` | Signification |
|---------------------------|---------------|
| `start` | Dates de debut (variables de decision) |
| `duration` | Durees des taches (donnees) |
| `resource` | Besoin en ressource de chaque tache (donnees) |
| `capacity` | Capacite maximale de la ressource (donnee) |

La contrainte `cumulative` garantit qu'a aucun instant la somme des ressources utilisees ne depasse la capacite. C'est l'equivalent de dizaines de contraintes binaires codees a la main.

> **Note technique** : en interne, le solveur utilise des algorithmes specialises (timetable, edge-finding, not-first/not-last) qui sont bien plus efficaces qu'une decomposition en contraintes elementaires.

In [ ]:
# Exemple circuit : mini-TSP
model_tsp = """
include "globals.mzn";

% 5 villes, matrice de distances
int: n = 5;
array[1..n, 1..n] of int: dist = 
  [| 0,  10, 15, 20, 25
   | 10,  0, 35, 25, 30
   | 15, 35,  0, 30, 20
   | 20, 25, 30,  0, 15
   | 25, 30, 20, 15,  0 |];

% Variables : next[i] = ville visitee apres la ville i
array[1..n] of var 1..n: next;

% Contrainte circuit : les next forment un cycle hamiltonien
constraint circuit(next);

% Objectif : minimiser la distance totale
var int: total_dist = sum(i in 1..n)(dist[i, next[i]]);
solve minimize total_dist;

output ["Tour: "] ++ 
       [show(i) ++ " -> " ++ show(next[i]) ++ "  " | i in 1..n] ++
       ["\\nDistance totale = ", show(total_dist), "\\n"];
"""

display_model(model_tsp, "TSP avec circuit")

result = run_minizinc_model(model_tsp)
if result is None and not MINIZINC_CLI_AVAILABLE:
    print("Resultat attendu :")
    print("  Tour : 1->2->4->5->3->1 (ou equivalent)")
    print("  Distance totale = 80")

### Interpretation : contrainte circuit pour le TSP

**Sortie obtenue** : un circuit hamiltonien de distance minimale.

| Element | Explication |
|---------|-------------|
| `array[1..n, 1..n] of int: dist` | Matrice de distances 2D (donnees) |
| `circuit(next)` | Force `next` a former un seul cycle passant par toutes les villes |
| `dist[i, next[i]]` | Distance de la ville $i$ a la suivante (indexation par variable) |

**Points cles** :
1. Tout le modele TSP tient en ~5 lignes declaratives
2. `circuit` est une contrainte globale tres puissante : elle elimine les sous-tours automatiquement
3. En CP-SAT, il faudrait ajouter manuellement des contraintes d'elimination de sous-tours (MTZ ou lazy constraints)

---

## 4. MiniZinc pour l'emploi du temps (~10 min)

Revenons au probleme d'emploi du temps vu dans [App-5-Timetabling](App-5-Timetabling.ipynb). Nous allons montrer comment MiniZinc separe elegamment le **modele** des **donnees**.

### Le probleme

Affecter des cours a des creneaux horaires et des salles en respectant :
- Chaque cours est assigne a exactement un creneau et une salle
- Un enseignant ne peut pas donner deux cours en meme temps
- Une salle ne peut pas accueillir deux cours en meme temps
- La capacite de la salle doit etre suffisante pour les etudiants

### Modele MiniZinc

In [ ]:
# Modele d'emploi du temps en MiniZinc
model_timetabling = """
include "globals.mzn";

% --- Parametres (seraient dans un fichier .dzn en production) ---
int: n_courses = 6;
int: n_slots = 5;        % creneaux disponibles
int: n_rooms = 3;        % salles disponibles
int: n_teachers = 3;

% Enseignant responsable de chaque cours
array[1..n_courses] of 1..n_teachers: teacher = [1, 1, 2, 2, 3, 3];

% Nombre d'etudiants par cours
array[1..n_courses] of int: students = [30, 25, 40, 20, 35, 15];

% Capacite de chaque salle
array[1..n_rooms] of int: room_cap = [35, 45, 25];

% --- Variables de decision ---
array[1..n_courses] of var 1..n_slots: slot;  % creneau de chaque cours
array[1..n_courses] of var 1..n_rooms: room;  % salle de chaque cours

% --- Contraintes ---

% C1 : Un enseignant ne donne pas 2 cours au meme creneau
constraint forall(i, j in 1..n_courses where i < j /\\ teacher[i] = teacher[j])(
    slot[i] != slot[j]
);

% C2 : Une salle n'accueille pas 2 cours au meme creneau
constraint forall(i, j in 1..n_courses where i < j)(
    slot[i] != slot[j] \\/ room[i] != room[j]
);

% C3 : La salle est assez grande
constraint forall(i in 1..n_courses)(
    students[i] <= room_cap[room[i]]
);

solve satisfy;

output ["Cours ", show(i), " -> creneau ", show(slot[i]),
        ", salle ", show(room[i]),
        " (prof=", show(teacher[i]), ", ", show(students[i]), " eleves)\\n"
        | i in 1..n_courses];
"""

display_model(model_timetabling, "Emploi du temps")

### Analyse du modele

Examinons les elements syntaxiques avances utilises dans ce modele.

| Syntaxe MiniZinc | Equivalent Python | Description |
|------------------|-------------------|-------------|
| `forall(i, j in 1..n where i < j)` | `for i in range(n): for j in range(i+1,n):` | Double boucle avec filtre |
| `teacher[i] = teacher[j]` | `teacher[i] == teacher[j]` | Test d'egalite (= dans les contraintes) |
| `/\\` | `and` | Conjonction logique |
| `\\/` | `or` | Disjonction logique |
| `room_cap[room[i]]` | `room_cap[room[i]]` | Indexation par variable de decision |

### Separation modele/donnees

En production, les donnees seraient dans un fichier `.dzn` separe :

```
% fichier instance1.dzn
n_courses = 6;
n_slots = 5;
n_rooms = 3;
n_teachers = 3;
teacher = [1, 1, 2, 2, 3, 3];
students = [30, 25, 40, 20, 35, 15];
room_cap = [35, 45, 25];
```

Le modele `.mzn` deviendrait generique : on pourrait resoudre n'importe quelle instance en changeant uniquement le fichier de donnees, sans toucher au modele.

In [ ]:
# Execution du modele d'emploi du temps
result = run_minizinc_model(model_timetabling)

if result is None and not MINIZINC_CLI_AVAILABLE:
    print("Resultat attendu (exemple) :")
    print("  Cours 1 -> creneau 1, salle 2 (prof=1, 30 eleves)")
    print("  Cours 2 -> creneau 2, salle 1 (prof=1, 25 eleves)")
    print("  Cours 3 -> creneau 1, salle 2 (prof=2, 40 eleves)")
    print("  Cours 4 -> creneau 3, salle 3 (prof=2, 20 eleves)")
    print("  Cours 5 -> creneau 2, salle 2 (prof=3, 35 eleves)")
    print("  Cours 6 -> creneau 1, salle 3 (prof=3, 15 eleves)")

### Comparaison : emploi du temps en CP-SAT vs MiniZinc

Comparons la taille et la lisibilite des deux approches pour le meme probleme.

In [ ]:
# Meme probleme d'emploi du temps en CP-SAT pour comparaison
def solve_timetabling_cpsat():
    """Resout l'emploi du temps avec OR-Tools CP-SAT."""
    if not ORTOOLS_AVAILABLE:
        return None
    
    n_courses = 6
    n_slots = 5
    n_rooms = 3
    n_teachers = 3
    teacher = [1, 1, 2, 2, 3, 3]  # index 0-based en Python
    students = [30, 25, 40, 20, 35, 15]
    room_cap = [35, 45, 25]
    
    model = cp_model.CpModel()
    
    # Variables
    slot = [model.NewIntVar(0, n_slots - 1, f'slot_{i}') for i in range(n_courses)]
    room = [model.NewIntVar(0, n_rooms - 1, f'room_{i}') for i in range(n_courses)]
    
    # C1 : enseignant ne donne pas 2 cours en meme temps
    for i in range(n_courses):
        for j in range(i + 1, n_courses):
            if teacher[i] == teacher[j]:
                model.Add(slot[i] != slot[j])
    
    # C2 : une salle ne peut accueillir qu'un cours par creneau
    for i in range(n_courses):
        for j in range(i + 1, n_courses):
            # slot[i] != slot[j] OR room[i] != room[j]
            # Equivalent : NOT(slot[i]==slot[j] AND room[i]==room[j])
            b_slot = model.NewBoolVar(f'same_slot_{i}_{j}')
            b_room = model.NewBoolVar(f'same_room_{i}_{j}')
            model.Add(slot[i] == slot[j]).OnlyEnforceIf(b_slot)
            model.Add(slot[i] != slot[j]).OnlyEnforceIf(b_slot.Not())
            model.Add(room[i] == room[j]).OnlyEnforceIf(b_room)
            model.Add(room[i] != room[j]).OnlyEnforceIf(b_room.Not())
            model.AddBoolOr([b_slot.Not(), b_room.Not()])
    
    # C3 : capacite de la salle suffisante
    for i in range(n_courses):
        for r in range(n_rooms):
            if students[i] > room_cap[r]:
                model.Add(room[i] != r)
    
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        for i in range(n_courses):
            s = solver.Value(slot[i]) + 1
            r = solver.Value(room[i]) + 1
            print(f"  Cours {i+1} -> creneau {s}, salle {r}"
                  f" (prof={teacher[i]}, {students[i]} eleves)")
        return True
    return None


print("Comparaison du nombre de lignes")
print("=" * 50)
print(f"MiniZinc  : ~25 lignes (modele complet avec output)")
print(f"CP-SAT    : ~45 lignes (sans compter imports/affichage)")
print(f"Ratio     : MiniZinc ~1.8x plus concis")
print()

if ORTOOLS_AVAILABLE:
    print("Solution CP-SAT :")
    solve_timetabling_cpsat()
else:
    print("[OR-Tools non disponible]")

### Interpretation : MiniZinc vs CP-SAT pour l'emploi du temps

| Aspect | MiniZinc | CP-SAT |
|--------|----------|--------|
| **Disjonction** (`\/`) | Native dans les contraintes | Necessite des variables booleennes intermediaires |
| **Indexation par variable** | `room_cap[room[i]]` direct | Necessite `AddElement` ou decomposition |
| **Separation donnees** | Fichier `.dzn` | Tout dans le code Python |
| **Portabilite solveur** | Un modele, N solveurs | Lie a OR-Tools |

**Points cles** :
1. La disjonction logique est un cas ou MiniZinc brille : `\/` est naturel, tandis que CP-SAT necessite des variables booleennes auxiliaires (reification)
2. L'indexation par variable de decision (`room_cap[room[i]]`) est geree nativement en MiniZinc
3. Pour les grands projets, la separation modele/donnees de MiniZinc facilite la maintenance

> **Quand preferer CP-SAT ?** Quand le probleme necessite une forte integration avec du code Python (callbacks, heuristiques personnalisees, post-traitement complexe) ou quand les performances brutes de CP-SAT sont critiques.

---

## 5. Piloter MiniZinc depuis Python (~8 min)

Le package Python `minizinc` permet d'integrer MiniZinc dans un workflow Python : creer des modeles, passer des donnees dynamiquement, et recuperer les resultats comme objets Python.

### Installation

```bash
pip install minizinc
```

Il faut aussi que MiniZinc soit installe sur le systeme (https://www.minizinc.org/).

### Workflow Python-MiniZinc

```python
import minizinc

# 1. Creer le modele
model = minizinc.Model()
model.add_string("...")

# 2. Choisir un solveur
solver = minizinc.Solver.lookup("gecode")

# 3. Creer une instance (modele + solveur)
instance = minizinc.Instance(solver, model)

# 4. Passer des donnees
instance["n"] = 8

# 5. Resoudre
result = instance.solve()

# 6. Exploiter les resultats
print(result["x"])  # valeur de la variable x
```

### Exemple : Sudoku 4x4 depuis Python

In [ ]:
# Sudoku 4x4 en MiniZinc, pilote depuis Python
model_sudoku4 = """
include "globals.mzn";

int: n = 4;  % taille du Sudoku (4x4)
int: sub = 2;  % taille des sous-blocs (2x2)

% Grille initiale : 0 = case vide
array[1..n, 1..n] of 0..n: init;

% Variables de decision
array[1..n, 1..n] of var 1..n: grid;

% Fixer les valeurs initiales
constraint forall(i, j in 1..n where init[i,j] > 0)(
    grid[i,j] = init[i,j]
);

% Lignes : toutes differentes
constraint forall(i in 1..n)(
    alldifferent([grid[i,j] | j in 1..n])
);

% Colonnes : toutes differentes
constraint forall(j in 1..n)(
    alldifferent([grid[i,j] | i in 1..n])
);

% Blocs 2x2 : toutes differentes
constraint forall(br, bc in 0..sub-1)(
    alldifferent([grid[br*sub+r, bc*sub+c] | r, c in 1..sub])
);

solve satisfy;

output [show(grid[i,j]) ++ if j == n then "\\n" else " " endif
        | i in 1..n, j in 1..n];
"""

display_model(model_sudoku4, "Sudoku 4x4 generique")

# Grille initiale (0 = vide)
grille_initiale = [
    [0, 0, 3, 0],
    [3, 0, 0, 2],
    [0, 3, 0, 0],
    [0, 0, 2, 0]
]

print("Grille initiale :")
for row in grille_initiale:
    print("  ", " ".join(str(v) if v > 0 else "." for v in row))

# Execution si le package minizinc est disponible
if MINIZINC_AVAILABLE:
    try:
        model = minizinc.Model()
        model.add_string(textwrap.dedent(model_sudoku4).strip())
        solver = minizinc.Solver.lookup("gecode")
        instance = minizinc.Instance(solver, model)
        
        # Passer la grille initiale
        instance["init"] = grille_initiale
        
        import datetime
        result = instance.solve(timeout=datetime.timedelta(seconds=10))
        
        if result.status.has_solution():
            print("\nSolution trouvee :")
            grid = result["grid"]
            for row in grid:
                print("  ", " ".join(str(v) for v in row))
        else:
            print(f"Statut : {result.status}")
    except Exception as e:
        print(f"Erreur : {e}")
else:
    print("\n[Package minizinc non disponible]")
    print("Solution attendue :")
    solution_attendue = [
        [2, 1, 3, 4],
        [3, 4, 1, 2],
        [1, 3, 4, 2],  
        [4, 2, 2, 1]   # Note : dependant du solveur
    ]
    print("  2 1 3 4")
    print("  3 4 1 2")
    print("  1 3 4 2")
    print("  4 2 2 1")

print("\nVoir la serie Sudoku pour des solveurs CSP complets :")
print("  -> ../../Sudoku/README.md")

### Interpretation : Sudoku en MiniZinc

**Sortie obtenue** : la grille 4x4 completee avec toutes les contraintes satisfaites.

| Element du modele | Explication |
|-------------------|-------------|
| `array[1..n, 1..n] of 0..n: init` | Parametre : grille initiale passee depuis Python |
| `where init[i,j] > 0` | Filtre : ne fixer que les cases non vides |
| `[grid[i,j] \| j in 1..n]` | Comprehension : extraire une ligne |
| `grid[br*sub+r, bc*sub+c]` | Indexation calculee pour les sous-blocs |

**Avantage Python + MiniZinc** : on peut generer des grilles en Python (aleatoirement, depuis un fichier, depuis une API) et les passer au modele MiniZinc. Le modele reste inchange.

> **Lien** : la serie [Sudoku](../../Sudoku/README.md) explore 6+ solveurs differents pour le Sudoku 9x9, incluant backtracking, AC-3, CP-SAT, et des approches hybrides.

### Alternative : appel en ligne de commande

Si le package Python n'est pas installe mais que le CLI `minizinc` est disponible, on peut ecrire le modele dans un fichier temporaire et l'executer via `subprocess`.

In [ ]:
# Demonstration de l'appel CLI (si disponible)
import tempfile
import os

# Modele simple pour le test CLI
model_simple = textwrap.dedent("""
var 1..5: a;
var 1..5: b;
constraint a + b = 6;
constraint a < b;
solve satisfy;
output ["a=", show(a), " b=", show(b), "\\n"];
""").strip()

if MINIZINC_CLI_AVAILABLE:
    # Ecrire dans un fichier temporaire
    with tempfile.NamedTemporaryFile(mode='w', suffix='.mzn', delete=False) as f:
        f.write(model_simple)
        tmp_path = f.name
    
    try:
        proc = subprocess.run(
            ['minizinc', '--solver', 'gecode', tmp_path],
            capture_output=True, text=True, timeout=10
        )
        print("Sortie CLI MiniZinc :")
        print(f"  {proc.stdout.strip()}")
        if proc.returncode != 0:
            print(f"  Erreur : {proc.stderr}")
    finally:
        os.unlink(tmp_path)
else:
    print("CLI minizinc non disponible.")
    print("Pour installer MiniZinc : https://www.minizinc.org/")
    print(f"\nModele :")
    print(model_simple)
    print(f"\nResultat attendu : a=1 b=5 (ou a=2 b=4)")

---

## 6. Comparaison : imperatif vs declaratif (~5 min)

Nous avons maintenant vu les deux paradigmes en action. Synthetisons les differences fondamentales.

### N-Reines : trois approches cote a cote

In [ ]:
# Approche 1 : Python pur (backtracking)
def solve_nqueens_pure_python(n):
    """N-Reines par backtracking en Python pur."""
    def is_safe(queens, row, col):
        for c, r in enumerate(queens):
            if r == row or abs(c - col) == abs(r - row):
                return False
        return True
    
    def solve(queens, col):
        if col == n:
            return list(queens)
        for row in range(n):
            if is_safe(queens, row, col):
                queens.append(row)
                result = solve(queens, col + 1)
                if result is not None:
                    return result
                queens.pop()
        return None
    
    return solve([], 0)


# Comparaison des trois approches
n = 12
results = []

# Python pur
start = time.time()
sol_py = solve_nqueens_pure_python(n)
t_py = (time.time() - start) * 1000
results.append({'approach': 'Python pur', 'time_ms': t_py, 'found': sol_py is not None})
print(f"Python pur   : {t_py:8.2f} ms  -> {sol_py is not None}")

# CP-SAT
if ORTOOLS_AVAILABLE:
    sol_cpsat, t_cpsat = solve_nqueens_cpsat(n)
    results.append({'approach': 'CP-SAT', 'time_ms': t_cpsat * 1000, 'found': sol_cpsat is not None})
    print(f"CP-SAT       : {t_cpsat*1000:8.2f} ms  -> {sol_cpsat is not None}")

# MiniZinc (si disponible)
if MINIZINC_AVAILABLE or MINIZINC_CLI_AVAILABLE:
    mzn_nq = f"""
    include "globals.mzn";
    int: n = {n};
    array[1..n] of var 1..n: q;
    constraint alldifferent(q);
    constraint alldifferent([q[i]+i | i in 1..n]);
    constraint alldifferent([q[i]-i | i in 1..n]);
    solve satisfy;
    output [show(q)];
    """
    start = time.time()
    result_mzn = run_minizinc_model(mzn_nq)
    t_mzn = (time.time() - start) * 1000
    results.append({'approach': 'MiniZinc', 'time_ms': t_mzn, 'found': result_mzn is not None})
    print(f"MiniZinc     : {t_mzn:8.2f} ms  -> {result_mzn is not None}")

print(f"\n{n}-Reines : toutes les approches trouvent une solution.")

### Tableau recapitulatif des paradigmes

In [ ]:
# Tableau comparatif detaille
comparison_data = {
    'Critere': [
        'Lignes de code (N-Reines)',
        'Lisibilite mathematique',
        'Flexibilite du solveur',
        'Integration Python',
        'Separation modele/donnees',
        'Debug et introspection',
        'Performance brute',
        'Courbe d\'apprentissage',
        'Cas d\'usage ideal'
    ],
    'Python pur': [
        '~20 lignes',
        'Faible',
        'Aucune (ad-hoc)',
        'Totale',
        'Non',
        'Complet (print, debugger)',
        'Lent pour gros problemes',
        'Facile (Python standard)',
        'Prototypage, pedagogie'
    ],
    'CP-SAT': [
        '~12 lignes',
        'Moyenne',
        'CP-SAT uniquement',
        'Excellente (API native)',
        'Non (code Python)',
        'Bon (solver stats, logs)',
        'Tres rapide',
        'Moyenne (API specifique)',
        'Production, performance'
    ],
    'MiniZinc': [
        '~5 lignes',
        'Haute',
        'Multi-solveur',
        'Via package python',
        'Oui (.mzn + .dzn)',
        'Limite (output, trace)',
        'Depend du solveur choisi',
        'Moyenne (syntaxe dediee)',
        'Modelisation, recherche'
    ]
}

# Affichage sous forme de tableau
print("Comparaison des paradigmes de resolution")
print("=" * 85)
header = f"{'Critere':<30} {'Python pur':<18} {'CP-SAT':<18} {'MiniZinc':<18}"
print(header)
print("-" * 85)

for i, critere in enumerate(comparison_data['Critere']):
    py_val = comparison_data['Python pur'][i]
    cpsat_val = comparison_data['CP-SAT'][i]
    mzn_val = comparison_data['MiniZinc'][i]
    print(f"{critere:<30} {py_val:<18} {cpsat_val:<18} {mzn_val:<18}")

print("=" * 85)

### Interpretation : choix du paradigme

**Quand utiliser chaque approche :**

| Situation | Recommandation | Raison |
|-----------|---------------|--------|
| Cours / pedagogie | Python pur ou MiniZinc | Comprendre les mecanismes |
| Prototypage rapide | MiniZinc | Concision, pas de boilerplate |
| Production Python | CP-SAT | Performance, integration |
| Recherche academique | MiniZinc | Portabilite entre solveurs |
| Application embarquee | CP-SAT ou solveur natif | Controle total |

**Points cles** :
1. MiniZinc est le meilleur choix pour **communiquer** un modele (articles, presentations, documentation)
2. CP-SAT est le meilleur choix pour **deployer** une solution en production Python
3. Les deux peuvent coexister : prototyper en MiniZinc, deployer en CP-SAT

> **A retenir** : le paradigme declaratif (MiniZinc) est a la programmation par contraintes ce que SQL est aux bases de donnees -- un langage de specification, pas d'implementation.

In [ ]:
# Visualisation : concision des approches
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graphique 1 : lignes de code par probleme
problems = ['N-Reines', 'Emploi du temps', 'Sudoku']
lines_python = [20, 60, 50]
lines_cpsat = [12, 45, 35]
lines_mzn = [5, 25, 20]

x_pos = np.arange(len(problems))
width = 0.25

axes[0].bar(x_pos - width, lines_python, width, label='Python pur', color='#2196F3')
axes[0].bar(x_pos, lines_cpsat, width, label='CP-SAT', color='#4CAF50')
axes[0].bar(x_pos + width, lines_mzn, width, label='MiniZinc', color='#FF9800')

axes[0].set_xlabel('Probleme')
axes[0].set_ylabel('Lignes de code')
axes[0].set_title('Concision du modele', fontweight='bold')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(problems)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Graphique 2 : radar des criteres (approximation en barres groupees)
criteria = ['Concision', 'Lisibilite', 'Portabilite', 'Integration\nPython', 'Performance']
scores_python = [2, 2, 1, 5, 2]
scores_cpsat = [3, 3, 2, 5, 5]
scores_mzn = [5, 5, 5, 3, 4]

x_crit = np.arange(len(criteria))

axes[1].bar(x_crit - width, scores_python, width, label='Python pur', color='#2196F3')
axes[1].bar(x_crit, scores_cpsat, width, label='CP-SAT', color='#4CAF50')
axes[1].bar(x_crit + width, scores_mzn, width, label='MiniZinc', color='#FF9800')

axes[1].set_xlabel('Critere')
axes[1].set_ylabel('Score (1-5)')
axes[1].set_title('Evaluation multi-criteres', fontweight='bold')
axes[1].set_xticks(x_crit)
axes[1].set_xticklabels(criteria, fontsize=9)
axes[1].set_ylim(0, 6)
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.suptitle('Comparaison des paradigmes de modelisation',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## 7. Exercices

### Exercice 1 : Carre magique en MiniZinc

Un **carre magique** d'ordre $n$ est une grille $n \times n$ contenant les entiers $1$ a $n^2$, ou la somme de chaque ligne, chaque colonne et les deux diagonales est la meme.

Pour $n = 3$, cette somme magique vaut $\frac{n(n^2+1)}{2} = 15$.

**Consigne** : completez le modele MiniZinc ci-dessous pour le carre magique $3 \times 3$.

In [ ]:
# Exercice 1 : Carre magique en MiniZinc

model_magic_square = """
include "globals.mzn";

int: n = 3;
int: magic_sum = n * (n * n + 1) div 2;  % = 15 pour n=3

array[1..n, 1..n] of var 1..n*n: grid;

% A COMPLETER :

% C1 : tous les entiers de 1 a n^2 sont differents
% constraint alldifferent(???);

% C2 : somme de chaque ligne = magic_sum
% constraint forall(i in 1..n)(???);

% C3 : somme de chaque colonne = magic_sum
% constraint forall(j in 1..n)(???);

% C4 : somme de la diagonale principale = magic_sum
% constraint ???;

% C5 : somme de l'anti-diagonale = magic_sum
% constraint ???;

solve satisfy;

output [show(grid[i,j]) ++ if j == n then "\\n" else " " endif
        | i in 1..n, j in 1..n];
"""

display_model(model_magic_square, "Exercice 1 : Carre magique (a completer)")

<details>
<summary><b>Solution exercice 1</b></summary>

```minizinc
include "globals.mzn";

int: n = 3;
int: magic_sum = n * (n * n + 1) div 2;

array[1..n, 1..n] of var 1..n*n: grid;

% C1 : tous differents
constraint alldifferent([grid[i,j] | i in 1..n, j in 1..n]);

% C2 : somme de chaque ligne
constraint forall(i in 1..n)(
    sum(j in 1..n)(grid[i,j]) = magic_sum
);

% C3 : somme de chaque colonne
constraint forall(j in 1..n)(
    sum(i in 1..n)(grid[i,j]) = magic_sum
);

% C4 : diagonale principale
constraint sum(i in 1..n)(grid[i,i]) = magic_sum;

% C5 : anti-diagonale
constraint sum(i in 1..n)(grid[i, n+1-i]) = magic_sum;

solve satisfy;
```

**Resultat attendu** (une des 8 solutions) :
```
2 7 6
9 5 1
4 3 8
```

</details>

### Exercice 2 : Coloration de graphe en MiniZinc

Reprenez le probleme de coloration de graphe de [App-2-GraphColoring](App-2-GraphColoring.ipynb) et ecrivez le modele MiniZinc correspondant.

Le graphe est le suivant (6 noeuds, 7 aretes) :
```
A -- B -- C
|    |    |
D -- E -- F
```

**Consigne** : ecrire un modele MiniZinc qui determine le nombre chromatique (nombre minimum de couleurs).

In [ ]:
# Exercice 2 : Coloration de graphe en MiniZinc

# A COMPLETER : ecrire le modele MiniZinc
model_coloring = """
% Coloration du graphe : A-B-C, A-D, B-E, C-F, D-E, E-F
% Objectif : minimiser le nombre de couleurs

% int: n_nodes = 6;
% int: max_colors = ???;

% A COMPLETER...

% solve minimize ???;
"""

display_model(model_coloring, "Exercice 2 : Coloration de graphe (a completer)")

<details>
<summary><b>Solution exercice 2</b></summary>

```minizinc
int: n_nodes = 6;
int: max_colors = 6;  % borne superieure

% Variables : couleur de chaque noeud
array[1..n_nodes] of var 1..max_colors: color;
% Nommage : 1=A, 2=B, 3=C, 4=D, 5=E, 6=F

% Aretes du graphe
constraint color[1] != color[2];  % A-B
constraint color[2] != color[3];  % B-C
constraint color[1] != color[4];  % A-D
constraint color[2] != color[5];  % B-E
constraint color[3] != color[6];  % C-F
constraint color[4] != color[5];  % D-E
constraint color[5] != color[6];  % E-F

% Nombre chromatique = max des couleurs utilisees
var 1..max_colors: n_colors = max(color);

% Cassage de symetrie : la couleur i apparait avant la couleur i+1
constraint color[1] = 1;  % le premier noeud prend toujours la couleur 1

solve minimize n_colors;

output ["Couleurs: ", show(color), "\n",
        "Nombre chromatique: ", show(n_colors), "\n"];
```

**Resultat attendu** : nombre chromatique = 2 (graphe biparti).

</details>

### Exercice 3 : VRP simplifie en MiniZinc

Le **Vehicle Routing Problem** (VRP) consiste a trouver des tournees optimales pour un ensemble de vehicules partant d'un depot et devant visiter des clients.

**Version simplifiee** : un seul vehicule, 5 clients + 1 depot (noeud 1), minimiser la distance totale.

**Consigne** : completez le modele en utilisant la contrainte `circuit`.

*Indice* : c'est essentiellement un TSP avec un depot fixe.

In [ ]:
# Exercice 3 : VRP simplifie en MiniZinc

model_vrp = """
include "globals.mzn";

% 6 noeuds : 1 = depot, 2..6 = clients
int: n = 6;
array[1..n, 1..n] of int: dist = 
  [| 0,  10, 15, 20, 10, 25
   | 10,  0, 20, 15, 10, 30
   | 15, 20,  0, 10, 25, 15
   | 20, 15, 10,  0, 20, 10
   | 10, 10, 25, 20,  0, 15
   | 25, 30, 15, 10, 15,  0 |];

% A COMPLETER :
% - Variables next[i] pour le successeur de chaque noeud
% - Contrainte circuit
% - Objectif : minimiser la distance totale

% array[1..n] of var 1..n: next;
% constraint ???;
% var int: total_dist = ???;
% solve minimize total_dist;
"""

display_model(model_vrp, "Exercice 3 : VRP simplifie (a completer)")

<details>
<summary><b>Solution exercice 3</b></summary>

```minizinc
include "globals.mzn";

int: n = 6;
array[1..n, 1..n] of int: dist = 
  [| 0,  10, 15, 20, 10, 25
   | 10,  0, 20, 15, 10, 30
   | 15, 20,  0, 10, 25, 15
   | 20, 15, 10,  0, 20, 10
   | 10, 10, 25, 20,  0, 15
   | 25, 30, 15, 10, 15,  0 |];

array[1..n] of var 1..n: next;

constraint circuit(next);

var int: total_dist = sum(i in 1..n)(dist[i, next[i]]);

solve minimize total_dist;

output ["Tour: depot"] ++
       [" -> " ++ show(next[i]) | i in 1..n] ++
       ["\nDistance: ", show(total_dist), "\n"];
```

**Note** : pour un vrai VRP multi-vehicules, il faudrait des noeuds depot dupliques ou une modelisation plus avancee.

</details>

---

## 8. Recapitulatif

### Resume de la syntaxe MiniZinc

| Element | Syntaxe | Exemple |
|---------|---------|--------|
| Parametre | `int: n = 8;` | Constante connue |
| Variable | `var 1..n: x;` | Inconnue a determiner |
| Tableau | `array[1..n] of var 1..n: q;` | Vecteur de variables |
| Contrainte | `constraint expr;` | Restriction |
| Quantificateur | `forall(i in 1..n)(...)` | Pour tout |
| Comprehension | `[f(i) \| i in 1..n]` | Tableau calcule |
| Satisfaction | `solve satisfy;` | Trouver une solution |
| Optimisation | `solve minimize obj;` | Trouver la meilleure |
| Sortie | `output [show(x)];` | Formatage du resultat |

### Contraintes globales vues

| Contrainte | Application | Avantage |
|------------|-------------|----------|
| `alldifferent` | Sudoku, N-Reines, coloration | Propagation specialisee (couplage biparti) |
| `cumulative` | Ordonnancement | Gestion des ressources en O(n log n) |
| `circuit` | TSP, VRP | Elimination automatique des sous-tours |

### Ce qu'il faut retenir

| Concept | Imperatif (CP-SAT) | Declaratif (MiniZinc) |
|---------|-------------------|----------------------|
| Philosophie | Decrire comment resoudre | Decrire quoi resoudre |
| Force principale | Performance, integration | Concision, portabilite |
| Faiblesse | Verbeux, lie au solveur | Moins de controle fin |
| Analogie | Programmation procedurale | SQL pour les contraintes |

### Et ensuite ?

- [App-9-EdgeDetection](App-9-EdgeDetection.ipynb) : optimisation par algorithmes genetiques pour la detection de contours
- [App-5-Timetabling](App-5-Timetabling.ipynb) : emploi du temps complet avec MiniZinc + OR-Tools
- Serie [Sudoku](../../Sudoku/README.md) : 6+ solveurs differents pour un meme probleme

### Ressources

- [MiniZinc Handbook](https://www.minizinc.org/doc-2.7.6/en/index.html) : documentation officielle
- [MiniZinc IDE](https://www.minizinc.org/ide/) : environnement de developpement dedie
- [MiniZinc Challenge](https://www.minizinc.org/challenge.html) : competition annuelle de solveurs
- Stuckey, P.J. et al. *MiniZinc: Towards a Standard CP Modelling Language*, CP 2007

---

**Navigation** : [<< App-7 Wordle](App-7-Wordle.ipynb) | [Index](../README.md) | [App-9 EdgeDetection >>](App-9-EdgeDetection.ipynb)